## DAY2通し課題
- 汎化誤差、正則化、SVM

### 1.DAY1作業内容の復習
#### ・ロジスティック回帰という手法でname,usd_goal_real,keikanissuuおよびmain_categoryを用いてstateを予測した。

In [26]:
#必要なライブラリのインポート
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split # ホールドアウト法に関する関数
from sklearn.model_selection import KFold # 交差検証法に関する関数
from sklearn.metrics import mean_absolute_error # 回帰問題における性能評価に関する関数

In [27]:
#DAY1のデータの読み込み
Kickstarter= pd.read_csv("Kickstarter_Day1.csv")
display(Kickstarter.head())

,name,state,usd_goal_real,keikanissuu,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,...,country_JP,country_LU,country_MX,"country_N,0""",country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,3,1,0.007656,34,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,8,1,-0.036530,19,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,1,-0.026160,29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9,1,-0.031598,27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,-0.037207,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### name=プロジェクト名の文字数
#### state=成功失敗のステータス
#### usd_goal_real=米国ドル版の目標公募金額(標準化してる)
#### keikanissuu=募集期間
#### main_category=大きな分類（ダミー変数にした）

### 2.交差検証（クロスバリデーション）法
#### ・交差検証法とは、データを複数のグループにわけ、テスト役と学習役を交代させていくことで少ないデータでも汎化誤差を評価する方法

In [31]:
X=Kickstarter.drop(["state"],axis=1)
y=Kickstarter["state"]

n_split=5
cross_valid_mae = 0
split_num = 1

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X.iloc[train_idx,:], y[train_idx] 
    X_test, y_test = X.iloc[test_idx,:], y[test_idx]  
    

    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    y_pred_test = clf.predict(X_train)
    
    accuracy=100 * accuracy_score(y_train, y_pred_test)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_pred_test)
    
    print("Fold %s"%split_num)
    print("accuracy = %s"%round(accuracy, 3))
    print("precision[1] = %s"%round(precision[1]*100, 3))
    print("recall[1] = %s"%round(recall[1]*100, 3))
    print("f1_score[1] = %s"%round(f1_score[1]*100, 3))
    split_num += 1
    
    

Fold 1
accuracy = 60.478
precision[1] = 50.811
recall[1] = 73.415
f1_score[1] = 60.056
Fold 2
accuracy = 64.496
precision[1] = 56.13
recall[1] = 55.014
f1_score[1] = 55.566
Fold 3
accuracy = 64.295
precision[1] = 55.523
recall[1] = 57.578
f1_score[1] = 56.532
Fold 4
accuracy = 64.961
precision[1] = 60.353
recall[1] = 38.27
f1_score[1] = 46.839
Fold 5
accuracy = 59.673
precision[1] = 50.11
recall[1] = 74.154
f1_score[1] = 59.806


In [29]:
Kickstarter.to_csv('Kickstarter_Day2.csv', index=False)

### 3.正則化
#### ・ホールドアウト法と正則化を行う。

In [56]:
X=Kickstarter.drop(["state"],axis=1).values
y=Kickstarter["state"].values
X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size = 0.1, random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=1234)

std_scaler = StandardScaler() 
X_train= std_scaler.fit_transform(X_train)
X_test= std_scaler.fit_transform(X_test)
X_final_test= std_scaler.fit_transform(X_final_test)


for i in [0.1, 0.01, 0.001,0.0001,0.00001]:
    clf = SGDClassifier(loss='log', penalty="l1", alpha=i,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)
    w0 = clf.intercept_[0]
    w = clf.coef_
    y_pred_test = clf.predict(X_train)
    accuracy=100 * accuracy_score(y_train, y_pred_test)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_pred_test)
    print("訓練データについて")
    print("accuracy = %s"%round(accuracy, 3))
    print("precision[1] = %s"%round(precision[1]*100, 3))
    print("recall[1] = %s"%round(recall[1]*100, 3))
    print("f1_score[1] = %s"%round(f1_score[1]*100, 3))
    y_pred_test = clf.predict(X_test)
    accuracy=100 * accuracy_score(y_test, y_pred_test)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred_test)
    print("テストデータについて")
    print("accuracy = %s"%round(accuracy, 3))
    print("precision[1] = %s"%round(precision[1]*100, 3))
    print("recall[1] = %s"%round(recall[1]*100, 3))
    print("f1_score[1] = %s"%round(f1_score[1]*100, 3))


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


訓練データについて
accuracy = 59.528
precision[1] = 0.0
recall[1] = 0.0
f1_score[1] = 0.0
テストデータについて
accuracy = 59.921
precision[1] = 0.0
recall[1] = 0.0
f1_score[1] = 0.0
訓練データについて
accuracy = 63.136
precision[1] = 60.286
recall[1] = 26.126
f1_score[1] = 36.454
テストデータについて
accuracy = 63.576
precision[1] = 60.537
recall[1] = 26.195
f1_score[1] = 36.567
訓練データについて
accuracy = 64.313
precision[1] = 59.3
recall[1] = 37.689
f1_score[1] = 46.086
テストデータについて
accuracy = 64.544
precision[1] = 59.176
recall[1] = 37.195
f1_score[1] = 45.679
訓練データについて
accuracy = 64.997
precision[1] = 60.237
recall[1] = 39.754
f1_score[1] = 47.898
テストデータについて
accuracy = 65.572
precision[1] = 61.109
recall[1] = 38.783
f1_score[1] = 47.451
訓練データについて
accuracy = 63.393
precision[1] = 58.205
recall[1] = 33.867
f1_score[1] = 42.819
テストデータについて
accuracy = 63.515
precision[1] = 57.912
recall[1] = 32.823
f1_score[1] = 41.899


In [46]:
clf = SGDClassifier(loss='log', penalty="none", max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
clf.fit(X_train, y_train)
w0 = clf.intercept_[0]
w = clf.coef_
y_pred_test = clf.predict(X_train)
accuracy=100 * accuracy_score(y_train, y_pred_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_pred_test)
print("訓練データについて")
print("accuracy = %s"%round(accuracy, 3))
print("precision[1] = %s"%round(precision[1]*100, 3))
print("recall[1] = %s"%round(recall[1]*100, 3))
print("f1_score[1] = %s"%round(f1_score[1]*100, 3))
y_pred_test = clf.predict(X_test)
accuracy=100 * accuracy_score(y_test, y_pred_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred_test)
print("テストデータについて")
print("accuracy = %s"%round(accuracy, 3))
print("precision[1] = %s"%round(precision[1]*100, 3))
print("recall[1] = %s"%round(recall[1]*100, 3))
print("f1_score[1] = %s"%round(f1_score[1]*100, 3))



訓練データについて
accuracy = 64.715
precision[1] = 58.438
recall[1] = 44.376
f1_score[1] = 50.445
テストデータについて
accuracy = 65.12
precision[1] = 58.766
recall[1] = 43.48
f1_score[1] = 49.981


In [51]:
clf = SGDClassifier(loss='log', penalty="none", max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
clf.fit(X_train, y_train)
w0 = clf.intercept_[0]
w = clf.coef_
y_pred_test = clf.predict(X_final_test)
accuracy=100 * accuracy_score(y_final_test, y_pred_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_final_test, y_pred_test)
print("最終テストデータについて")
print("accuracy = %s"%round(accuracy, 3))
print("precision[1] = %s"%round(precision[1]*100, 3))
print("recall[1] = %s"%round(recall[1]*100, 3))
print("f1_score[1] = %s"%round(f1_score[1]*100, 3))


最終テストデータについて
accuracy = 64.849
precision[1] = 60.555
recall[1] = 34.71
f1_score[1] = 44.127


In [55]:
clf = clf = SGDClassifier(loss='log', penalty="l1", alpha=0.0001,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
clf.fit(X_train, y_train)
w0 = clf.intercept_[0]
w = clf.coef_
y_pred_test = clf.predict(X_final_test)
accuracy=100 * accuracy_score(y_final_test, y_pred_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_final_test, y_pred_test)
print("最終テストデータについて")
print("accuracy = %s"%round(accuracy, 3))
print("precision[1] = %s"%round(precision[1]*100, 3))
print("recall[1] = %s"%round(recall[1]*100, 3))
print("f1_score[1] = %s"%round(f1_score[1]*100, 3))

最終テストデータについて
accuracy = 65.048
precision[1] = 62.156
recall[1] = 32.207
f1_score[1] = 42.429


### 4.SVMの使用

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
# SVMの実行
C = 5
kernel = "rbf"
gamma = 1
clf = SVC(C=C, kernel=kernel, gamma=gamma)
clf.fit(X_train, y_train)
w0 = clf.intercept_[0]
w = clf.coef_
y_pred_test = clf.predict(X_final_test)
accuracy=100 * accuracy_score(y_final_test, y_pred_test)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_final_test, y_pred_test)
print("最終テストデータについて")
print("accuracy = %s"%round(accuracy, 3))
print("precision[1] = %s"%round(precision[1]*100, 3))
print("recall[1] = %s"%round(recall[1]*100, 3))
print("f1_score[1] = %s"%round(f1_score[1]*100, 3))

In [ ]:
#グリッドサーチによるパラメータの最適化
parameters = {'kernel':['rbf'], 'C':[0.1, 1, 10], 'gamma':[0.1, 1, 10]} 
model = SVC() 
clf = GridSearchCV(model, parameters, cv=3,)
clf.fit(X_train, y_train)
print(clf.best_params_, clf.best_score_)